In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import numpy as np
import warnings

code_dir = os.path.join(os.path.abspath(os.pardir), 'prepare_dataset')

sys.path.append(code_dir)
sys.path.append(os.path.join(code_dir, 'struct2depth'))
sys.path.append(os.path.join(code_dir, 'tfoptflow/tfoptflow'))

warnings.simplefilter('ignore')

In [ ]:
image_directory = None
depth_directory = None
video_path = None
json_path = None
csv_path = None
txt_path = None

optical_flow_estimator_name = 'pwc'
depth_estimator_name = 'struct2depth'
#depth_estimator_name = 'senet'

computation_kwargs = dict(
    cuda_visible_devices=3,
)

In [ ]:
from dataset_builder import *

In [ ]:
# build_from = DIRECTORY
# build_from = VIDEO
# build_from = CSV
# build_from = KITTI
# build_from = DISCOMAN
build_from = TUM

In [ ]:
if build_from == VIDEO:
    sequence_directory = 'test_build_from_video'
    video_path = '/Vol1/dbstore/datasets/saic_odometry/07/t_video5316726383492203313.mp4'
    DatasetBuilder = VideoDatasetBuilder

if build_from == DIRECTORY:
    sequence_directory = 'test_build_from_dir'
    image_directory = '/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/00/image_2'
    DatasetBuilder = ImagesDatasetBuilder
    
if build_from == CSV:
    sequence_directory = 'test_build_from_csv'
    csv_path = ''
    DatasetBuilder = CSVDatasetBuilder

if build_from == KITTI:
    sequence_directory = 'test_build_kitti'
    image_directory = '/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/00/image_2'
    depth_directory = '/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/00/depth_2'
    depth_estimator_name = None
    DatasetBuilder = KITTIDatasetBuilder
    
if build_from == DISCOMAN:
    sequence_directory = 'test_build_discoman'
    json_path = '/Vol1/dbstore/datasets/renderbox/v10.5/traj/output/train/0000/0_traj.json'
    depth_estimator_name = None
    DatasetBuilder = DISCOMANDatasetBuilder
    
if build_from == TUM:
    sequence_directory = 'test_build_tum'
    txt_path = '/Vol1/dbstore/datasets/tum_rgbd_flow/data/rgbd_dataset_freiburg2_coke/groundtruth.txt'
    DatasetBuilder = TUMDatasetBuilder

In [ ]:
estimate_optical_flow = optical_flow_estimator_name is not None
estimate_depth = depth_estimator_name is not None

### Initialize

In [ ]:
builder = DatasetBuilder(sequence_directory, 
                         build_from=build_from,
                         image_directory=image_directory,
                         depth_directory=depth_directory,
                         video_path=video_path,
                         json_path=json_path,
                         csv_path=csv_path,
                         txt_path=txt_path,
                         mode=DatasetBuilder.TEST,
                         estimate_depth=estimate_depth,
                         estimate_optical_flow=estimate_optical_flow,
                         optical_flow_estimator_name=optical_flow_estimator_name,
                         depth_estimator_name=depth_estimator_name,
                         memory_safe=True,
                         **computation_kwargs)
builder

### Build

In [ ]:
builder._configure()

In [ ]:
if builder.build_from == VIDEO:
    print(builder.video_parser)

if builder.build_from == DIRECTORY:
    pass
    
if builder.build_from == CSV:
    pass

if builder.build_from == KITTI:
    pass

if builder.build_from == DISCOMAN:
    print(builder.parser)
    
if builder.build_from == TUM:
    print(builder.parser)    

In [ ]:
builder

In [ ]:
print(builder.dataframe.shape)
builder.dataframe.head()

In [ ]:
!nvidia-smi

In [ ]:
builder.build() # "Do everything" big red button

In [ ]:
!nvidia-smi

In [ ]:
print(builder.dataframe.shape)
builder.dataframe.head()

### Visualize

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def load(path):
    if path.endswith('npy'):
        return np.load(path)
    return cv2.imread(path)[:, :, ::-1]


def visualize_depth(depth):
    plt.imshow(depth[..., 0], cmap='RdBu')
    plt.show()

def visualize_optical_flow(optical_flow):
    fig, ax = plt.subplots(ncols=2, figsize=(15, 5))
    ax[0].imshow(optical_flow[..., 0], cmap='RdBu')
    ax[1].imshow(optical_flow[..., 1], cmap='RdBu')
    plt.show()

In [ ]:
index = 0
visualize_depth(load(builder.dataframe.path_to_depth[index]))
visualize_optical_flow(load(builder.dataframe.path_to_optical_flow[index]))